In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Encoded


*   Jumlah kolom 17 (id, text) + 15 kolom top-N tag



In [ ]:
import pandas as pd, ast
from sklearn.preprocessing import MultiLabelBinarizer
unique_tags = set()
for chunk in pd.read_csv("/content/drive/MyDrive/StackOverflow_Data/new/taglist_top15_bersih.csv", chunksize=20000):
    chunk["tag_list"] = chunk["tag_list"].apply(ast.literal_eval)
    for tags in chunk["tag_list"]:
        unique_tags.update(tags)

unique_tags = sorted(unique_tags)
print(unique_tags)

# Simpan biar gak hitung ulang
import json
with open("/content/drive/MyDrive/StackOverflow_Data/new/unique_tags.json", "w") as f:
    json.dump(unique_tags, f)

['android', 'c#', 'c++', 'css', 'html', 'java', 'javascript', 'node.js', 'pandas', 'php', 'python', 'python-3.x', 'r', 'reactjs', 'sql']


In [ ]:
import json, pandas as pd, ast, gc
from sklearn.preprocessing import MultiLabelBinarizer

with open("/content/drive/MyDrive/StackOverflow_Data/new/unique_tags.json") as f:
    unique_tags = json.load(f)

mlb = MultiLabelBinarizer(classes=unique_tags)
mlb.fit([unique_tags])

input_path = "/content/drive/MyDrive/StackOverflow_Data/new/taglist_top15_bersih.csv"
output_path = "/content/drive/MyDrive/StackOverflow_Data/new/taglist_top15_encoded.csv"
chunksize = 20000

first = True
for i, chunk in enumerate(pd.read_csv(input_path, chunksize=chunksize)):
    chunk["tag_list"] = chunk["tag_list"].apply(ast.literal_eval)
    y = mlb.transform(chunk["tag_list"])
    y_df = pd.DataFrame(y, columns=mlb.classes_)
    encoded_chunk = pd.concat([chunk[["Id", "text"]].reset_index(drop=True), y_df], axis=1)
    encoded_chunk.to_csv(output_path, mode="a", index=False, header=first)
    first = False
    print(f"Chunk {i+1} selesai")
    del chunk, y, y_df, encoded_chunk
    gc.collect()


Chunk 1 selesai
Chunk 2 selesai
Chunk 3 selesai
Chunk 4 selesai
Chunk 5 selesai
Chunk 6 selesai
Chunk 7 selesai
Chunk 8 selesai
Chunk 9 selesai
Chunk 10 selesai
Chunk 11 selesai
Chunk 12 selesai
Chunk 13 selesai
Chunk 14 selesai
Chunk 15 selesai
Chunk 16 selesai
Chunk 17 selesai
Chunk 18 selesai
Chunk 19 selesai
Chunk 20 selesai
Chunk 21 selesai
Chunk 22 selesai
Chunk 23 selesai
Chunk 24 selesai
Chunk 25 selesai
Chunk 26 selesai
Chunk 27 selesai
Chunk 28 selesai
Chunk 29 selesai
Chunk 30 selesai
Chunk 31 selesai
Chunk 32 selesai
Chunk 33 selesai
Chunk 34 selesai
Chunk 35 selesai
Chunk 36 selesai
Chunk 37 selesai
Chunk 38 selesai
Chunk 39 selesai
Chunk 40 selesai
Chunk 41 selesai
Chunk 42 selesai
Chunk 43 selesai
Chunk 44 selesai
Chunk 45 selesai
Chunk 46 selesai
Chunk 47 selesai
Chunk 48 selesai
Chunk 49 selesai
Chunk 50 selesai
Chunk 51 selesai
Chunk 52 selesai
Chunk 53 selesai
Chunk 54 selesai
Chunk 55 selesai
Chunk 56 selesai
Chunk 57 selesai
Chunk 58 selesai
Chunk 59 selesai
Chunk 

In [ ]:
import pandas as pd

encoded_path = "/content/drive/MyDrive/StackOverflow_Data/new/taglist_top15_encoded.csv"

df = pd.read_csv(encoded_path, nrows=5)
print("Kolom yang tersedia:", df.columns.tolist())
print("\nJumlah kolom total:", len(df.columns))
print("\nContoh isi data:")
print(df.head(3))


Kolom yang tersedia: ['Id', 'text', 'android', 'c#', 'c++', 'css', 'html', 'java', 'javascript', 'node.js', 'pandas', 'php', 'python', 'python-3.x', 'r', 'reactjs', 'sql']

Jumlah kolom total: 17

Contoh isi data:
         Id                                               text  android  c#  \
0  59551209  android in app purchase service connection is ...        1   0   
1  59551210  parse key val list where space has dual meanin...        0   0   
2  59551216  c unable to return json object of large data? ...        0   1   

   c++  css  html  java  javascript  node.js  pandas  php  python  python-3.x  \
0    0    0     0     0           0        0       0    0       0           0   
1    0    0     0     0           0        0       0    0       1           0   
2    0    0     0     0           1        0       0    0       0           0   

   r  reactjs  sql  
0  0        0    0  
1  0        0    0  
2  0        0    0  


#Train & Test

##Split 90/10


---
ini mengacu pada jurnal acuan


In [ ]:
import pandas as pd
import numpy as np
import os

input_path = "/content/drive/MyDrive/StackOverflow_Data/new/taglist_top15_encoded.csv"
train_path = "/content/drive/MyDrive/StackOverflow_Data/new/split_90-10/train.csv"
test_path = "/content/drive/MyDrive/StackOverflow_Data/new/split_90-10/test.csv"

chunksize = 200000
train_ratio = 0.9

# hapus file lama jika ada
for f in [train_path, test_path]:
    if os.path.exists(f):
        os.remove(f)

print("Mulai split bertahap (90/10)")

total_train, total_test = 0, 0
first = True

for i, chunk in enumerate(pd.read_csv(input_path, chunksize=chunksize)):
    # hitung jumlah baris untuk train di chunk ini
    n_train = int(len(chunk) * train_ratio)

    # acak baris
    chunk = chunk.sample(frac=1, random_state=42).reset_index(drop=True)

    # bagi 90/10
    train_chunk = chunk.iloc[:n_train]
    test_chunk = chunk.iloc[n_train:]

    # simpan ke file, append
    train_chunk.to_csv(train_path, mode="a", index=False, header=first)
    test_chunk.to_csv(test_path, mode="a", index=False, header=first)
    first = False

    total_train += len(train_chunk)
    total_test += len(test_chunk)

    print(f"Chunk {i+1} selesai. Train total: {total_train}, Test total: {total_test}")

print("\nSplit selesai!")
print(f"Data train tersimpan di: {train_path}")
print(f"Data test tersimpan di: {test_path}")
print(f"Total train: {total_train}, Total test: {total_test}")


Mulai split bertahap (90/10)
Chunk 1 selesai. Train total: 180000, Test total: 20000
Chunk 2 selesai. Train total: 360000, Test total: 40000
Chunk 3 selesai. Train total: 540000, Test total: 60000
Chunk 4 selesai. Train total: 720000, Test total: 80000
Chunk 5 selesai. Train total: 900000, Test total: 100000
Chunk 6 selesai. Train total: 1080000, Test total: 120000
Chunk 7 selesai. Train total: 1260000, Test total: 140000
Chunk 8 selesai. Train total: 1440000, Test total: 160000
Chunk 9 selesai. Train total: 1620000, Test total: 180000
Chunk 10 selesai. Train total: 1800000, Test total: 200000
Chunk 11 selesai. Train total: 1980000, Test total: 220000
Chunk 12 selesai. Train total: 2160000, Test total: 240000
Chunk 13 selesai. Train total: 2340000, Test total: 260000
Chunk 14 selesai. Train total: 2520000, Test total: 280000
Chunk 15 selesai. Train total: 2700000, Test total: 300000
Chunk 16 selesai. Train total: 2880000, Test total: 320000
Chunk 17 selesai. Train total: 3060000, Test 

In [ ]:
import pandas as pd

train_path = "/content/drive/MyDrive/StackOverflow_Data/new/split_90-10/train.csv"
test_path = "/content/drive/MyDrive/StackOverflow_Data/new/split_90-10/test.csv"

# Cek jumlah baris
train_rows = sum(1 for _ in open(train_path)) - 1
test_rows = sum(1 for _ in open(test_path)) - 1
total = train_rows + test_rows

train_ratio = train_rows / total * 100
test_ratio = test_rows / total * 100

print(f"Jumlah total data: {total:,}")
print(f"Jumlah train: {train_rows:,} ({train_ratio:.2f}%)")
print(f"Jumlah test : {test_rows:,} ({test_ratio:.2f}%)")


Jumlah total data: 3,431,399
Jumlah train: 3,088,259 (90.00%)
Jumlah test : 343,140 (10.00%)
